# Timeline

<img src="https://i.ibb.co/KKVVDjW/Image-Tittle.jpg" alt="Image-Tittle" border="0">

<img src="https://i.ibb.co/HKtKPy6/announcments.jpg" alt="announcments" border="0">

# Declaration

* According to MoHFW https://www.mohfw.gov.in/ some new cases have been reassigned states as per latest information
* Confirmed cases includes both Indian and Foriegn citizen
* Cured cases includes cured, discharged and migrated patients 

## Data Source
* https://www.mohfw.gov.in/


### GitHub Repository for this Notebook
<a href="https://github.com/sreyaz01/covid-19-india-data-analysis"><img src="https://i.ibb.co/B3vvzTy/0-s-Y-XTIBzlfd2zskq.png" alt="0-s-Y-XTIBzlfd2zskq" border="0" width="300" height="100"></a>

In [1]:
#!conda install -c conda-forge cufflinks-py

# Libraries

In [2]:
## utility libraries
from IPython.core.display import HTML
from datetime import datetime
from datetime import timedelta


# storing and anaysis
import pandas as pd
import geopandas as gpd
import numpy as np

#Visualization Libraries
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from plotly.offline import init_notebook_mode,plot,iplot

import folium
import seaborn as sns

import cufflinks as cf

# Warning
import warnings
warnings.filterwarnings('ignore')


print('Pandas Version' , pd.__version__)
print('Matplotlib Version' , matplotlib.__version__)
print('Plotly Version' , plotly.__version__)
print('Seaborn Version' , sns.__version__)

C:\Users\sreya\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



Pandas Version 1.0.1
Matplotlib Version 3.2.1
Plotly Version 4.5.4
Seaborn Version 0.9.0


In [3]:
# setting up some setting for libraries
%matplotlib inline
plt.rcParams['figure.figsize'] = 17,8
pyo.init_notebook_mode(connected=True)
cf.go_offline()

#style.use('ggplot')

In [4]:
# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow

# Dataset 

In [5]:
%ls C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\covid19-corona-virus-india-dataset

 Volume in drive C has no label.
 Volume Serial Number is 0821-B132

 Directory of C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\covid19-corona-virus-india-dataset

25-04-2020  22:38    <DIR>          .
25-04-2020  22:38    <DIR>          ..
27-04-2020  21:31             8,923 cases_over_time_flourish.csv
27-04-2020  21:30            68,040 complete.csv
10-04-2020  16:57         1,219,997 patients_data.csv
24-04-2020  23:26               731 pop2018.csv
               4 File(s)      1,297,691 bytes
               2 Dir(s)  259,668,369,408 bytes free


In [6]:
#importing data
df = pd.read_csv(r'C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\covid19-corona-virus-india-dataset\complete.csv',
                parse_dates = ['Date'])

df.tail()

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Latitude,Longitude,Death,Total Confirmed cases
1330,2020-04-27,Telengana,0,0,280,18.1124,79.0193,26,1002
1331,2020-04-27,Tripura,0,0,2,23.9408,91.9882,0,2
1332,2020-04-27,Uttar Pradesh,0,0,335,26.8467,80.9462,31,1955
1333,2020-04-27,Uttarakhand,0,0,33,30.0668,79.0193,0,51
1334,2020-04-27,West Bengal,0,0,105,22.9868,87.8550,20,649


In [7]:
df.columns

Index(['Date', 'Name of State / UT', 'Total Confirmed cases (Indian National)',
       'Total Confirmed cases ( Foreign National )',
       'Cured/Discharged/Migrated', 'Latitude', 'Longitude', 'Death',
       'Total Confirmed cases'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1335 entries, 0 to 1334
Data columns (total 9 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   Date                                        1335 non-null   datetime64[ns]
 1   Name of State / UT                          1335 non-null   object        
 2   Total Confirmed cases (Indian National)     1335 non-null   int64         
 3   Total Confirmed cases ( Foreign National )  1335 non-null   int64         
 4   Cured/Discharged/Migrated                   1335 non-null   int64         
 5   Latitude                                    1335 non-null   float64       
 6   Longitude                                   1335 non-null   float64       
 7   Death                                       1335 non-null   int64         
 8   Total Confirmed cases                       1335 non-null   int64         
dtypes: dateti

## State Map data

In [9]:
# shape files
# map_data = gpd.read_file(r'C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\india-district-wise-shape-files\output.shp')

# map_data.head()

In [10]:
#State Wise Grouping Data
# states = map_data.dissolve(by='statename').reset_index()
# states.head()

In [11]:
# states['statename'] = states['statename'].str.replace('&', 'and')
# states['statename'] = states['statename'].str.replace('NCT of ', '')
# states['statename'] = states['statename'].str.replace('Chhatisgarh', 'Chhattisgarh')
# states['statename'] = states['statename'].str.replace('Orissa', 'Odisha')
# states['statename'] = states['statename'].str.replace('Pondicherry', 'Puducherry')

# states['statename'].unique()

# Preprocessing


## Cleaning

In [12]:
df_clean = df[['Date', 'Name of State / UT', 'Latitude', 'Longitude', 'Total Confirmed cases', 'Death', 'Cured/Discharged/Migrated']]
df_clean.columns = ['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths', 'Cured']

df_clean['Date'] = df_clean['Date'].dt.date

df_clean['Active'] = df_clean['Confirmed']-(df_clean['Deaths']+df_clean['Cured'])
df_clean['Mortality Rate'] = df_clean['Deaths']/df_clean['Confirmed']
df_clean['Recovery Rate'] = df_clean['Cured']/df_clean['Confirmed']
df_clean.tail()

,Date,State/UT,Latitude,Longitude,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
1330,2020-04-27,Telengana,18.1124,79.0193,1002,26,280,696,0.025948,0.279441
1331,2020-04-27,Tripura,23.9408,91.9882,2,0,2,0,0.000000,1.000000
1332,2020-04-27,Uttar Pradesh,26.8467,80.9462,1955,31,335,1589,0.015857,0.171355
1333,2020-04-27,Uttarakhand,30.0668,79.0193,51,0,33,18,0.000000,0.647059
1334,2020-04-27,West Bengal,22.9868,87.8550,649,20,105,524,0.030817,0.161787


## Finding Latest Data Only

In [13]:
latest = df_clean[df_clean['Date']==max(df_clean['Date'])]


total_confirm = latest['Confirmed'].sum()
total_active = latest['Active'].sum()
total_cured = latest['Cured'].sum()
total_death = latest['Deaths'].sum()


now  = datetime.now().strftime("%B %d, %Y")

print(u"\u2022",f'Total Number of Confirmed Covid 2019 Cases across India till date ({now}):', total_confirm)
print(u"\u2022",f'Total Number of Active Cases till date ({now}):', total_active)
print(u"\u2022",f'Total Number of Cured Cases across India till date ({now}):', total_cured)
print(u"\u2022",f'Total Number of Deaths across India till date ({now}):', total_death)

• Total Number of Confirmed Covid 2019 Cases across India till date (April 27, 2020): 28160
• Total Number of Active Cases till date (April 27, 2020): 20912
• Total Number of Cured Cases across India till date (April 27, 2020): 6362
• Total Number of Deaths across India till date (April 27, 2020): 886


In [14]:
tm = latest.melt(id_vars="Date", value_vars=['Active', 'Deaths', 'Cured'])
tm.head()

,Date,variable,value
0,2020-04-27,Active,22
1,2020-04-27,Active,911
2,2020-04-27,Active,0
3,2020-04-27,Active,8
4,2020-04-27,Active,219


In [15]:
fig = px.treemap(tm, path=["variable"], values="value",height=250, width=800,
                 color_discrete_sequence=[act, rec, dth], title='Latest Stats')

fig.data[0].textinfo = 'label+value+text'
fig.show()

In [16]:
latest.head()

,Date,State/UT,Latitude,Longitude,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
1303,2020-04-27,Andaman and Nicobar Islands,11.7401,92.6586,33,0,11,22,0.000000,0.333333
1304,2020-04-27,Andhra Pradesh,15.9129,79.7400,1177,31,235,911,0.026338,0.199660
1305,2020-04-27,Arunachal Pradesh,28.2180,94.7278,1,0,1,0,0.000000,1.000000
1306,2020-04-27,Assam,26.2006,92.9376,36,1,27,8,0.027778,0.750000
1307,2020-04-27,Bihar,25.0961,85.3131,277,2,56,219,0.007220,0.202166


In [17]:
temp = latest.groupby(by = ['State/UT']).sum()

temp.tail()

,Latitude,Longitude,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
State/UT,,,,,,,,
Telengana,18.1124,79.0193,1002,26,280,696,0.025948,0.279441
Tripura,23.9408,91.9882,2,0,2,0,0.000000,1.000000
Uttar Pradesh,26.8467,80.9462,1955,31,335,1589,0.015857,0.171355
Uttarakhand,30.0668,79.0193,51,0,33,18,0.000000,0.647059
West Bengal,22.9868,87.8550,649,20,105,524,0.030817,0.161787


In [18]:
temp = temp[['Confirmed','Deaths','Cured','Active','Mortality Rate','Recovery Rate']]
temp.sort_values('Confirmed',ascending=False,inplace = True)
#temp.head()

temp.style\
    .background_gradient(cmap="Blues", subset=['Active','Confirmed'])\
    .background_gradient(cmap="Greens", subset=['Cured', 'Recovery Rate'])\
    .background_gradient(cmap="Reds", subset=['Deaths', 'Mortality Rate'])

,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
State/UT,,,,,,
Maharashtra,8068,342,1188,6538,0.042390,0.147248
Gujarat,3301,151,313,2837,0.045744,0.094820
Delhi,2918,54,877,1987,0.018506,0.300548
Rajasthan,2185,41,518,1626,0.018764,0.237071
Madhya Pradesh,2168,106,302,1760,0.048893,0.139299
Uttar Pradesh,1955,31,335,1589,0.015857,0.171355
Tamil Nadu,1885,24,1020,841,0.012732,0.541114
Andhra Pradesh,1177,31,235,911,0.026338,0.199660
Telengana,1002,26,280,696,0.025948,0.279441


In [19]:
temp.columns

Index(['Confirmed', 'Deaths', 'Cured', 'Active', 'Mortality Rate',
       'Recovery Rate'],
      dtype='object')

In [20]:
#Visualization
temp_1 = temp[['Confirmed', 'Deaths', 'Cured']]

temp_1.iplot(kind = 'bar',xTitle= 'State/UT' , yTitle='Numbers of Cases',mode = 'markers+lines',
            title = f'Cases State Wise on {now}')

In [21]:
temp_2 = temp[['Mortality Rate','Recovery Rate']]
               
temp_2.iplot(kind ='scatter',xTitle='State/UT',yTitle='Avrage',title = f'Mortality and Recovery Rate on {now}',
             mode = 'markers', size = 5)

In [22]:
# Date wise data visualization whole country

temp = df_clean.groupby(by = ['Date']).sum()
temp.drop(['Latitude','Longitude','Mortality Rate','Recovery Rate'],axis=1,inplace=True)

temp.tail()

,Confirmed,Deaths,Cured,Active
Date,,,,
2020-04-23,21393,681,4258,16454
2020-04-24,23077,718,4749,17610
2020-04-25,24893,779,5210,18904
2020-04-26,26605,826,5914,19865
2020-04-27,28160,886,6362,20912


In [23]:
temp.iplot(title = 'Covid-19 Growth in India', yTitle='Cases',size=5,mode='markers+lines')

## No. of New Cases EveryDay After 50

In [24]:
#df_clean.columns

In [25]:
cases_df = df_clean.groupby('Date')['Confirmed', 'Deaths'].sum()

filt_cnf = (cases_df['Confirmed'] >= 50)

temp = cases_df[filt_cnf].diff().dropna()

col_y = ['Confirmed','Deaths']
colr = [cnf,dth]

for i,x in enumerate(col_y):
    temp.iplot(kind = 'scatter',mode = "markers+lines" ,size = 5,y = col_y[i],color=colr[i],yTitle=col_y[i],title=f'New {col_y[i]} Cases after Crossing 50 Confirmed Cases')

## Top 10 States

In [26]:
top_10 = latest.groupby(by = ['State/UT']).agg({'Confirmed': 'sum', 'Deaths': 'sum', 'Cured' : 'sum', 'Active' : 'sum'})\
                .nlargest(10,['Confirmed','Deaths','Cured','Active'])

top_10

,Confirmed,Deaths,Cured,Active
State/UT,,,,
Maharashtra,8068,342,1188,6538
Gujarat,3301,151,313,2837
Delhi,2918,54,877,1987
Rajasthan,2185,41,518,1626
Madhya Pradesh,2168,106,302,1760
Uttar Pradesh,1955,31,335,1589
Tamil Nadu,1885,24,1020,841
Andhra Pradesh,1177,31,235,911
Telengana,1002,26,280,696


In [27]:
#Creating Figures
plot_c = px.bar(top_10.sort_values('Confirmed') ,x="Confirmed",y = top_10.sort_values('Confirmed').index,
               text='Confirmed', orientation='h', color_discrete_sequence = [cnf])

plot_d = px.bar(top_10.sort_values('Deaths'),x="Deaths",y = top_10.sort_values('Deaths').index,
               text='Deaths', orientation='h', color_discrete_sequence = [dth])

plot_r = px.bar(top_10.sort_values('Cured'),x="Cured",y = top_10.sort_values('Cured').index,
               text='Cured', orientation='h', color_discrete_sequence = [rec])

plot_a = px.bar(top_10.sort_values('Active'),x="Active",y = top_10.sort_values('Active').index,
               text='Active', orientation='h', color_discrete_sequence = [act])


# plot
fig = make_subplots(rows=2, cols=2, shared_xaxes=False, horizontal_spacing=0.14, vertical_spacing=0.08,
                    subplot_titles=('Confirmed cases', 'Deaths reported', 'Recovered', 'Active cases'))

fig.add_trace(plot_c['data'][0],row=1, col=1)
fig.add_trace(plot_d['data'][0],row=1, col=2)
fig.add_trace(plot_r['data'][0],row=2, col=1)
fig.add_trace(plot_a['data'][0],row=2, col=2)

fig.update_layout(height=600 ,title_text="Top 10 States ")


# Cases Rises in State Over Time

https://app.flourish.studio/visualisation/1977187/edit

In [28]:
HTML('<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/1977187" data-url="https://flo.uri.sh/visualisation/1977187/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>')

In [29]:
df_clean.columns

Index(['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths',
       'Cured', 'Active', 'Mortality Rate', 'Recovery Rate'],
      dtype='object')

In [30]:
#Deaths,Cured ,Active Cases Date Wise

col = ['Deaths','Cured','Active']

for i,val in enumerate(col):
    p_df = pd.pivot_table(df_clean,index  = 'Date', values = val, columns ='State/UT').fillna(0).astype('int').reset_index()
    p_df.iplot(x = 'Date' ,title = col[i], xTitle = 'Date',yTitle = 'Cases')

# Geographical Map

In [31]:
latest.columns

Index(['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths',
       'Cured', 'Active', 'Mortality Rate', 'Recovery Rate'],
      dtype='object')

In [32]:
geo_map = folium.Map([20.5937,78.9629],zoom_start=4,tiles ='cartodbpositron' )

for lat,long,active,deaths,cured,name in zip(latest['Latitude'],latest['Longitude'],\
                                             latest['Active'],latest['Deaths'],\
                                             latest['Cured'],latest['State/UT']):

    folium.CircleMarker([lat,long],radius=active*0.01\
                       ,tooltip = (f'''<strong>name</strong>: {str(name).capitalize()} <br>
                               <strong>Active</strong>: {str(active)}<br>
                               <strong>Deaths</strong>: {str(deaths)}<br>
                               <strong>Cured</strong>: {str(cured)}<br>''')\
                       ,color = 'red',fill_color = 'red',fill_opacity=0.3).add_to(geo_map)

geo_map


## No of State affected Over Time

In [33]:
grp_states = df_clean.groupby('Date')['State/UT']
affected_states = grp_states.unique().apply(len).values
#affected_states


dates = grp_states.unique().apply(len).index
#dates

In [34]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=dates, y=[36 for i in range(len(affected_states))], 
                         mode='lines', name='Total no. of States+UT', 
                         line = dict(color='#222831', dash='longdashdot')))

fig.add_trace(go.Scatter(x=dates, y=affected_states, hoverinfo='x+y',
                         mode='lines', name='No. of affected States+UT', 
                         line = dict(color='#c70039')))

fig.update_layout(title='No. of affected States/UT over Time', 
                  xaxis_title='Dates', yaxis_title='No. of affected States/UT')
fig.show()

## Confirmed Vs Deaths

In [35]:
latest_cnf_dth = latest[latest['Confirmed']>10]
latest_cnf_dth = latest_cnf_dth[['State/UT','Confirmed','Deaths']]

px.scatter(latest_cnf_dth,x = 'Confirmed', y = 'Deaths', color ='State/UT', size = "Confirmed",log_x=True, title = 'Confirmed vs Deaths')



### Estimate Population 2018 vs Confirmed Cases
<br>
<li> <b> Data Source :</b> </li>
    http://statisticstimes.com/demographics/population-of-indian-states.php

In [36]:
# Reading Popolation Data

pop2018 = pd.read_csv(r'C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\covid19-corona-virus-india-dataset/pop2018.csv')

pop2018.rename(columns = {'State': 'State/UT'},inplace = True)
pop2018.dtypes

pop2018['State/UT'].replace('Telangana', 'Telengana', inplace=True)
pop2018['State/UT'].replace('Jammu & Kashmir', 'Jammu and Kashmir', inplace=True)
pop2018['State/UT'].replace('A.& N.Islands', 'Andaman and Nicobar Islands', inplace=True)

In [37]:
states_l = []
for state in latest['State/UT'].tolist():
    latest_state_list = pop2018['State/UT'].tolist()
    if state in latest_state_list:
        pass
    else:
        print(state, 'is not in latest DataFrame')
        
        
len(states_l)

Ladakh is not in latest DataFrame


0

In [38]:
pop2018.shape

(36, 2)

In [39]:
latest.shape

(32, 10)

In [40]:
merge = pd.merge(latest,pop2018, on = 'State/UT')

In [41]:
merge

,Date,State/UT,Latitude,Longitude,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate,2018
0,2020-04-27,Andaman and Nicobar Islands,11.7401,92.6586,33,0,11,22,0.000000,0.333333,419978
1,2020-04-27,Andhra Pradesh,15.9129,79.7400,1177,31,235,911,0.026338,0.199660,52883163
2,2020-04-27,Arunachal Pradesh,28.2180,94.7278,1,0,1,0,0.000000,1.000000,1528296
3,2020-04-27,Assam,26.2006,92.9376,36,1,27,8,0.027778,0.750000,34586234
4,2020-04-27,Bihar,25.0961,85.3131,277,2,56,219,0.007220,0.202166,119461013
5,2020-04-27,Chandigarh,30.7333,76.7794,30,0,17,13,0.000000,0.566667,1126705
6,2020-04-27,Chhattisgarh,21.2787,81.8661,37,0,32,5,0.000000,0.864865,28566990
7,2020-04-27,Delhi,28.7041,77.1025,2918,54,877,1987,0.018506,0.300548,18345784
8,2020-04-27,Goa,15.2993,74.1240,7,0,7,0,0.000000,1.000000,1542750
9,2020-04-27,Gujarat,22.2587,71.1924,3301,151,313,2837,0.045744,0.094820,63907200


In [42]:
Ladakh = 290,492
Andaman and Nicobar Islands = 380,581

SyntaxError: invalid syntax (<ipython-input-42-0070294af5f0>, line 2)